### Introduction
This notebook demonstrates how I built highly accurate shot predictions for the MyDigitalTennisCoach app.  
I show the code and working towards the predictions, and how I combined them for the best results.  

I share my approaches and reasoning at different points and conclude with ways in which I could enhance this.  

There are elements which lay outside of building the predictions, such as the data creation and conversion.  
These will be covered in separate workbooks.

In [1]:
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme("chesterish")

In [1]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
# pd.set_option("display.max_columns", 500)
# pd.set_option("display.max_rows", 500)
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## To Dos
Run everything first - understand what have done and then change it

Why does some of my data get imported as mixed types - which and fix  
Do the models change when I just randomly split the test and train groups?  
Can do the train & test split and create some visualisations - like I already have to create these  

Elab on why use merror vs mlogloss - in function

Elaborate on the need for validation & eval - is it because of model stacking?

### More Experiments
Does FH BH only preds give a better indication - simple? Require labelling?

## Source Data
The data was gathered via IMUs, IoT devices that measure accelerameter and gyroscope data at frequencies of 50 - 100Hz.  
This data has been preprocessed, meaning it has been identified as a shot and cut to the core data and labelled.  
The method by which is has been identified as a shot will be built into a separate notebook.

In [2]:
# read in all data sets and combine
d1 = pd.read_csv("D:/OneDrive/DataSci/Tennis/03_Modelling/DataSets/Watch_Q122_DataAndPreds.csv")
d1["Source"] = "WatchQ122"
d2 = pd.read_csv("D:/OneDrive/DataSci/Tennis/03_Modelling/DataSets/Xsens_EvalData.csv")
d2["Source"] = "XsensEval"
d3 = pd.read_csv("D:/OneDrive/DataSci/Tennis/03_Modelling/DataSets/Xsens_ModelData.csv")
d3["Source"] = "XsensModel"
d4 = pd.read_csv("D:/OneDrive/DataSci/Tennis/03_Modelling/DataSets/Watch_Q2Data.csv")
d4["Source"] = "WatchQ222"

d5 = pd.read_csv("D:/OneDrive/DataSci/Tennis/03_Modelling/DataSets/Xsens_ServeEvalData.csv")
d5["Source"] = "XsensServeEval"

df = pd.concat([d1,d2,d3,d4, d5] , axis = 0)
#restrict to broader key variables
df2 = df[d4.columns.to_list()]
df2 = df2[df2.Shot.notnull()]
df2 = df2[df2.Shot.isin(["FH", "BH", "Serve", "Slice", "Volley", "BH_Slice", "FH_Slice", "BHS", "OH", "Overhead", "FHS"])]
print(df2.shape)

C:\Users\james\AppData\Local\Temp\ipykernel_6968\1462714270.py:4: DtypeWarning: Columns (739,740,741,742,743,749) have mixed types. Specify dtype option on import or set low_memory=False.
  d2 = pd.read_csv("D:/OneDrive/DataSci/Tennis/03_Modelling/DataSets/Xsens_EvalData.csv")


(8606, 387)


In [3]:
#reset the index as the data is built on concats
df2 = df2.drop("Key", axis=1).reset_index(drop = True)
df2 = df2.reset_index().rename(columns = {"index": "origindex"})
print(df2.shape)
print(len(df2.groupby("origindex")["Shot"].count()))

(8606, 387)
8606


## A brief description of the data:  
There are 8606 shots that have been labelled, and each shot has 387 columns.  

366 of these are variables which can be used for prediction.  
The others are labels and other models which have been added to this data.  

There are 3 direction vectors measured by the devices, (X, Y & Z) for the accelerometer and gyroscope data.  
For each shot I found the strike point, and took the 30 milliseconds prior to this and the 30 millseconds post.  
This results in 61 measurements per measurement type, per directional plane.  
61 * 3 * 3 = 366

In [4]:
# There is a separate prediction for Serves and this notebook is focused on predictions for the other shots

# remove the serve data and create cleanshot which simplifies and aligns the labelling
df3 = df2[df2.Shot != "Serve"]
df3["CleanShot"] = np.where(df3.Shot.isin(["Slice", "BH_Slice", "FH_Slice", "BHS", "FHS"]), "Slice",
                           np.where(df3.Shot.isin(["OH", "Overhead"]), "OH", df3.Shot))
df3.groupby(["CleanShot", "Shot"])["Acc_X_01"].count()

CleanShot  Shot    
BH         BH          1833
FH         FH          3498
OH         OH            70
           Overhead      38
Slice      BHS          101
           BH_Slice     247
           FHS           37
           FH_Slice     113
           Slice        742
Volley     Volley       348
Name: Acc_X_01, dtype: int64

## Creating Training, Test, Validation & Eval Training Sets

This data is sourced from a number of players of different abilities.  
To ensure that the bias in technique and ability are not included in the models, each group includes a mix of those players.

In [5]:
df3["SourceHL"] = np.where(df3.Source.isin(["WatchQ122", "WatchQ222"]), "Watch", "Xsens")
df3["SourceHL2"] = np.where(df3.Source.isin(["WatchQ122", "WatchQ222"]), 1, 0)

df3["ModGroup"] = np.where(df3.Who.isin(["JJO", "SH","MH", "KS", "FS", "TL", "KD", "Sammy", "PS", "RO", "Frank", "Leo", "LM", "LeoMueller"]), "Mod",
                          np.where(df3.Who.isin(["JT", "PP_byEar", "KL", "SV", "RE", "RJ", "Roman" ]), "Validation", 
                          np.where(df3.Who.isin(["Toby_byEar","SVG", "LY", "JD", "Jimmy", "Kevin", "CH" ]), "Eval","Unassigned" )))

#generate split views to ensure that the mix is roughly right across the groups
splits = df3.groupby(["ModGroup","CleanShot"])["Acc_X_01"].count().reset_index().rename(columns = {"Acc_X_01": "Freq"})
splits_tot = df3.groupby("ModGroup")["Acc_X_01"].count().reset_index().rename(columns = {"Acc_X_01": "Totals"})
splits2 = pd.merge(splits, splits_tot, on = "ModGroup", how = "left" )
splits2["Prop"] = splits2.Freq / splits2.Totals
splits2

,ModGroup,CleanShot,Freq,Totals,Prop
0,Eval,BH,253,820,0.308537
1,Eval,FH,391,820,0.476829
2,Eval,OH,37,820,0.045122
3,Eval,Slice,103,820,0.125610
4,Eval,Volley,36,820,0.043902
5,Mod,BH,1304,5276,0.247157
6,Mod,FH,2676,5276,0.507202
7,Mod,OH,47,5276,0.008908
8,Mod,Slice,991,5276,0.187832
9,Mod,Volley,258,5276,0.048901


Looking at CleanShot == FH (Forehand) group, we can see from the Prop column, this represents 48% of the eval data, 51% in the Mod group, and 46% in the validation group.  
Ensuring these splits are similar across groups means I can evaluate the performance within by shot and at the ModGroup level.

In [6]:
# To avoid the possiblity of overfit, the mod data test is split to mod & test
df3["ModTestSplit"] = np.where(df3.Who.isin([ "MH", "LM", "Leo", "LeoMueller", "Frank", "TL"]), "modTest", "modMod")

#generate split views to ensure that the mix is roughly right across the groups
splits = df3[df3.ModGroup == "Mod"].groupby(["ModTestSplit","CleanShot"])["Acc_X_01"].count().reset_index().rename(columns = {"Acc_X_01": "Freq"})
splits_tot = df3[df3.ModGroup == "Mod"].groupby("ModTestSplit")["Acc_X_01"].count().reset_index().rename(columns = {"Acc_X_01": "Totals"})
splits2 = pd.merge(splits, splits_tot, on = "ModTestSplit", how = "left" )
splits2["Prop"] = splits2.Freq / splits2.Totals
splits2

,ModTestSplit,CleanShot,Freq,Totals,Prop
0,modMod,BH,912,3759,0.242618
1,modMod,FH,1837,3759,0.488694
2,modMod,OH,27,3759,0.007183
3,modMod,Slice,826,3759,0.219739
4,modMod,Volley,157,3759,0.041766
5,modTest,BH,392,1517,0.258405
6,modTest,FH,839,1517,0.553065
7,modTest,OH,20,1517,0.013184
8,modTest,Slice,165,1517,0.108767
9,modTest,Volley,101,1517,0.066579


In [7]:
#create labels for the data to predict against
a = {}
a["BH"] = 0
a["Slice"] = 1
a["FH"] = 2
a["Volley"] = 3
a["OH"] = 4

    
df3["Label_Num"] = df3.CleanShot.apply(lambda x: a[x])

In [8]:
def mod_shotid(dinput, eta,start1, fin1, start2, fin2, start3, fin3, start4, fin4, start5, fin5, start6, fin6 ):
    y=dinput[["Label_Num", "CleanShot","origindex","Who", "When","Shot", "PredShot", "Source", "ModTestSplit","SourceHL2"]]

    #adjust in i
    X1 = dinput.iloc[:, start1:fin1]
    X2 = dinput.iloc[:, start2:fin2]
    X3 = dinput.iloc[:, start3:fin3]
    X4 = dinput.iloc[:, start4:fin4]
    X5 = dinput.iloc[:, start5:fin5]
    X6 = dinput.iloc[:, start6:fin6]
    X7 = dinput[["ModTestSplit"]]
    
    X = pd.concat([X1, X2, X3, X4, X5, X6, X7], axis = 1)
#     print(X.head())
#     X_train, X_test, y_train1,y_test1=train_test_split(X,y, test_size=0.3, random_state=785)

    X_train = X[X.ModTestSplit == "modMod"].drop("ModTestSplit", axis = 1)
    X_test = X[X.ModTestSplit == "modTest"].drop("ModTestSplit", axis = 1)
#     print(X_train.head())
    y_train1 = y[y.ModTestSplit == "modMod"]
    y_test1 = y[y.ModTestSplit == "modTest"]
    y_test = y_test1[["Label_Num"]]
    y_train = y_train1[["Label_Num"]]

    dtrain = xgb.DMatrix(X_train,label=y_train)
    dtest = xgb.DMatrix(X_test,label=y_test)
    params={
            'max_depth':6,
        'min_child_weight': 4,
        'eta':eta,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
    #     "scale_pos_weight" : 8, #change me
        # Other parameters
        'eval_metric' : "merror",
        'objective':'multi:softprob',
        "num_class":5,
        'seed':123        
    }
    num_boost_round = 999
    mod=xgb.train(params,
                 dtrain,
                 num_boost_round=num_boost_round,
                 evals=[(dtest, "Test")],
                 early_stopping_rounds=30)
    
    
    results= y_test1[["CleanShot", "Label_Num","Who", "origindex"]].reset_index()

    probs = mod.predict(dtest)
    probs2 = pd.DataFrame({'A': probs[:, 0], 'B': probs[:, 1], 'C': probs[:, 2], 'D': probs[:, 3],
                          'E': probs[:, 4]})
    results=pd.concat([results, probs2],axis=1)

    # results["Correct"]= np.where( results.Label3 == results.preds, 1, 0)
    results['Max_Col'] = results.iloc[:,-5:].idxmax(axis=1)
    results['Max'] = results.iloc[:,-6:-1].max(axis=1)

    b = {}
    b["A"] = "BH"
    b["B"] = "Slice"
    b["C"] = "FH"
    b["D"] = "Volley"
    b["E"] = "OH"
#


    results["Preds"] = results.Max_Col.apply(lambda x: b[x])
    results["Correct"] = np.where( results.Preds == results.CleanShot, 1, 0)
    print("CROSS TAB VIEW")
    print(pd.crosstab(results.CleanShot,results.Preds))
    print()
    print("PREDS Accuracy")

    print(results.groupby("Preds")["Correct"].agg({"count","sum","mean"}))
    print("ACTUAL Accuracy")
    print(results.groupby("CleanShot")["Correct"].agg({"count","sum","mean"}))
    return mod, results, y_test1, dtest, X_test
    

def gen_results(mod, X_test, valid, evalu):
    valid_y = valid[["Label_Num", "CleanShot","Who", "When","Shot", "PredShot", "Source", "ModTestSplit","SourceHL2",'Slice_Res_prob', 'Slice_Res_pred',
       'BH_2040Focus_prob', 'BH_2040Focus_pred', 'Combined_OldPred', "ServeProbs"]]
    
    evalu_y = evalu[["Label_Num", "CleanShot","Who", "When","Shot", "PredShot", "Source", "ModTestSplit","SourceHL2",'Slice_Res_prob', 'Slice_Res_pred',
       'BH_2040Focus_prob', 'BH_2040Focus_pred', 'Combined_OldPred', "ServeProbs"]]
    
    
    v_results= valid_y[["CleanShot", "Label_Num","Who", 'Slice_Res_prob', 'Slice_Res_pred',
       'BH_2040Focus_prob', 'BH_2040Focus_pred', 'Combined_OldPred', "ServeProbs"]].reset_index()
    valid_x = valid[X_test.columns.to_list()]
    probs = mod.predict(xgb.DMatrix(valid_x))
    probs2 = pd.DataFrame({'A': probs[:, 0], 'B': probs[:, 1], 'C': probs[:, 2], 'D': probs[:, 3],
                          'E': probs[:, 4]})
    v_results=pd.concat([v_results, probs2],axis=1)

    # results["Correct"]= np.where( results.Label3 == results.preds, 1, 0)
    v_results['Max_Col'] = v_results.iloc[:,-5:].idxmax(axis=1)
    v_results['Max'] = v_results.iloc[:,-6:-1].max(axis=1)

    b = {}
    b["A"] = "BH"
    b["B"] = "Slice"
    b["C"] = "FH"
    b["D"] = "Volley"
    b["E"] = "OH"
#


    v_results["Preds"] = v_results.Max_Col.apply(lambda x: b[x])
    v_results["Correct"] = np.where( v_results.Preds == v_results.CleanShot, 1, 0)
    
    
    evalu_x = evalu[X_test.columns.to_list()]
    
    e_results= evalu_y[["CleanShot", "Label_Num","Who", 'Slice_Res_prob', 'Slice_Res_pred',
       'BH_2040Focus_prob', 'BH_2040Focus_pred', 'Combined_OldPred', "ServeProbs"]].reset_index()
    evalu_x = evalu[X_test.columns.to_list()]
    eprobs = mod.predict(xgb.DMatrix(evalu_x))
    eprobs2 = pd.DataFrame({'A': eprobs[:, 0], 'B': eprobs[:, 1], 'C': eprobs[:, 2], 'D': eprobs[:, 3],
                          'E': eprobs[:, 4]})
    e_results=pd.concat([e_results, eprobs2],axis=1)

    # results["Correct"]= np.where( results.Label3 == results.preds, 1, 0)
    e_results['Max_Col'] = e_results.iloc[:,-5:].idxmax(axis=1)
    e_results['Max'] = e_results.iloc[:,-6:-1].max(axis=1)

    e_results["Preds"] = e_results.Max_Col.apply(lambda x: b[x])
    e_results["Correct"] = np.where( e_results.Preds == e_results.CleanShot, 1, 0)

    
#     print(valid_y.head())
    
    print(color.YELLOW+ color.BOLD+color.UNDERLINE+"Results of Validation Set" +color.END)
    print(v_results.groupby("CleanShot")["Correct"].agg({"count","sum","mean"}))
    print(v_results.groupby("Preds")["Correct"].agg({"count","sum","mean"}))
    print(pd.crosstab(v_results.CleanShot, v_results["Preds"]))
    print()
    
    print(color.YELLOW+ color.BOLD+color.UNDERLINE+"Results of Eval Set" +color.END)
    print(e_results.groupby("CleanShot")["Correct"].agg({"count","sum","mean"}))
    print(e_results.groupby("Preds")["Correct"].agg({"count","sum","mean"}))
    print(pd.crosstab(e_results.CleanShot, e_results["Preds"]))
    print()
    return v_results, e_results

In [32]:
mod_0_40, res_0_40, y_test_0_40, dtest_0_40, X_test_0_40 = mod_shotid(df3[df3.ModGroup == "Mod"], 0.05, 14, 54, 75, 115, 136, 176, 197, 237, 258, 298, 319, 359 )

[0]	Test-merror:0.27093
[1]	Test-merror:0.24588
[2]	Test-merror:0.24192
[3]	Test-merror:0.24061
[4]	Test-merror:0.24061
[5]	Test-merror:0.23797
[6]	Test-merror:0.23533
[7]	Test-merror:0.23863
[8]	Test-merror:0.23533
[9]	Test-merror:0.23665
[10]	Test-merror:0.23270
[11]	Test-merror:0.23336
[12]	Test-merror:0.23270
[13]	Test-merror:0.23006
[14]	Test-merror:0.22742
[15]	Test-merror:0.22676
[16]	Test-merror:0.22742
[17]	Test-merror:0.22347
[18]	Test-merror:0.22544
[19]	Test-merror:0.22676
[20]	Test-merror:0.23006
[21]	Test-merror:0.22544
[22]	Test-merror:0.22281
[23]	Test-merror:0.22413
[24]	Test-merror:0.22347
[25]	Test-merror:0.22413
[26]	Test-merror:0.22281
[27]	Test-merror:0.22149
[28]	Test-merror:0.22479
[29]	Test-merror:0.22347
[30]	Test-merror:0.22215
[31]	Test-merror:0.22149
[32]	Test-merror:0.22347
[33]	Test-merror:0.22544
[34]	Test-merror:0.22347
[35]	Test-merror:0.22281
[36]	Test-merror:0.22479
[37]	Test-merror:0.22413
[38]	Test-merror:0.22479
[39]	Test-merror:0.22215
[40]	Test-

In [33]:
mod_10_40, res_10_40, y_test_10_40, dtest_10_40, X_test_10_40 = mod_shotid(df3[df3.ModGroup == "Mod"], 0.05, 24, 54, 85, 115, 146, 176, 207, 237, 268, 298, 329, 359 )

[0]	Test-merror:0.26170
[1]	Test-merror:0.24588
[2]	Test-merror:0.25049
[3]	Test-merror:0.23863
[4]	Test-merror:0.22874
[5]	Test-merror:0.22479
[6]	Test-merror:0.22413
[7]	Test-merror:0.22610
[8]	Test-merror:0.22808
[9]	Test-merror:0.22544
[10]	Test-merror:0.22413
[11]	Test-merror:0.22610
[12]	Test-merror:0.22479
[13]	Test-merror:0.22479
[14]	Test-merror:0.22281
[15]	Test-merror:0.22149
[16]	Test-merror:0.22610
[17]	Test-merror:0.22215
[18]	Test-merror:0.22017
[19]	Test-merror:0.22017
[20]	Test-merror:0.22149
[21]	Test-merror:0.22017
[22]	Test-merror:0.22017
[23]	Test-merror:0.22083
[24]	Test-merror:0.21819
[25]	Test-merror:0.22083
[26]	Test-merror:0.21885
[27]	Test-merror:0.21885
[28]	Test-merror:0.21819
[29]	Test-merror:0.21753
[30]	Test-merror:0.21753
[31]	Test-merror:0.21951
[32]	Test-merror:0.21819
[33]	Test-merror:0.21951
[34]	Test-merror:0.21885
[35]	Test-merror:0.21688
[36]	Test-merror:0.21688
[37]	Test-merror:0.21622
[38]	Test-merror:0.21622
[39]	Test-merror:0.21622
[40]	Test-

In [11]:
mod_20_40, res_20_40, y_test_20_40, dtest_20_40, X_test_20_40 = mod_shotid(df3[df3.ModGroup == "Mod"], 0.05, 34, 54, 95, 115, 156, 176, 217, 237, 278, 298, 339, 359 )

[0]	Test-merror:0.27093
[1]	Test-merror:0.24061
[2]	Test-merror:0.23006
[3]	Test-merror:0.22544
[4]	Test-merror:0.21688
[5]	Test-merror:0.22017
[6]	Test-merror:0.21753
[7]	Test-merror:0.21556
[8]	Test-merror:0.21622
[9]	Test-merror:0.21490
[10]	Test-merror:0.21556
[11]	Test-merror:0.21424
[12]	Test-merror:0.21490
[13]	Test-merror:0.21358
[14]	Test-merror:0.21424
[15]	Test-merror:0.21292
[16]	Test-merror:0.21424
[17]	Test-merror:0.21292
[18]	Test-merror:0.21358
[19]	Test-merror:0.21622
[20]	Test-merror:0.21556
[21]	Test-merror:0.21358
[22]	Test-merror:0.21622
[23]	Test-merror:0.21556
[24]	Test-merror:0.21688
[25]	Test-merror:0.21490
[26]	Test-merror:0.21490
[27]	Test-merror:0.21160
[28]	Test-merror:0.21094
[29]	Test-merror:0.21028
[30]	Test-merror:0.21028
[31]	Test-merror:0.20897
[32]	Test-merror:0.20897
[33]	Test-merror:0.20897
[34]	Test-merror:0.20765
[35]	Test-merror:0.20897
[36]	Test-merror:0.21094
[37]	Test-merror:0.21028
[38]	Test-merror:0.20962
[39]	Test-merror:0.20831
[40]	Test-

In [10]:
mod_20_40, res_20_40, y_test_20_40, dtest_20_40, X_test_20_40 = mod_shotid(df3[df3.ModGroup == "Mod"], 0.3, 34, 54, 95, 115, 156, 176, 217, 237, 278, 298, 339, 359 )

[0]	Test-merror:0.27093
[1]	Test-merror:0.24654
[2]	Test-merror:0.24522
[3]	Test-merror:0.22874
[4]	Test-merror:0.22281
[5]	Test-merror:0.22149
[6]	Test-merror:0.22413
[7]	Test-merror:0.22413
[8]	Test-merror:0.22610
[9]	Test-merror:0.21951
[10]	Test-merror:0.21688
[11]	Test-merror:0.21622
[12]	Test-merror:0.21556
[13]	Test-merror:0.21556
[14]	Test-merror:0.21226
[15]	Test-merror:0.20765
[16]	Test-merror:0.20567
[17]	Test-merror:0.20962
[18]	Test-merror:0.21292
[19]	Test-merror:0.21226
[20]	Test-merror:0.21688
[21]	Test-merror:0.21028
[22]	Test-merror:0.20303
[23]	Test-merror:0.20171
[24]	Test-merror:0.20040
[25]	Test-merror:0.20303
[26]	Test-merror:0.20633
[27]	Test-merror:0.20699
[28]	Test-merror:0.20369
[29]	Test-merror:0.20435
[30]	Test-merror:0.20765
[31]	Test-merror:0.20369
[32]	Test-merror:0.20369
[33]	Test-merror:0.20369
[34]	Test-merror:0.20105
[35]	Test-merror:0.20303
[36]	Test-merror:0.20237
[37]	Test-merror:0.19908
[38]	Test-merror:0.19974
[39]	Test-merror:0.20105
[40]	Test-

In [98]:
mod_20_50, res_20_50, y_test_20_50, dtest_20_50, X_test_20_50 = mod_shotid(df3[df3.ModGroup == "Mod"], 0.05, 34, 64, 95, 125, 156, 186, 217, 247, 278, 308, 339, 369 )

[0]	Test-merror:0.27027
[1]	Test-merror:0.23863
[2]	Test-merror:0.22940
[3]	Test-merror:0.23533
[4]	Test-merror:0.23204
[5]	Test-merror:0.22808
[6]	Test-merror:0.22347
[7]	Test-merror:0.22281
[8]	Test-merror:0.21753
[9]	Test-merror:0.21556
[10]	Test-merror:0.21226
[11]	Test-merror:0.21753
[12]	Test-merror:0.21622
[13]	Test-merror:0.21292
[14]	Test-merror:0.21358
[15]	Test-merror:0.21490
[16]	Test-merror:0.21424
[17]	Test-merror:0.21358
[18]	Test-merror:0.21490
[19]	Test-merror:0.21622
[20]	Test-merror:0.21358
[21]	Test-merror:0.21094
[22]	Test-merror:0.21028
[23]	Test-merror:0.21028
[24]	Test-merror:0.20633
[25]	Test-merror:0.20765
[26]	Test-merror:0.20567
[27]	Test-merror:0.20699
[28]	Test-merror:0.20699
[29]	Test-merror:0.20567
[30]	Test-merror:0.20831
[31]	Test-merror:0.20962
[32]	Test-merror:0.20897
[33]	Test-merror:0.21028
[34]	Test-merror:0.21028
[35]	Test-merror:0.20897
[36]	Test-merror:0.20962
[37]	Test-merror:0.21094
[38]	Test-merror:0.21160
[39]	Test-merror:0.21028
[40]	Test-

In [36]:
#experiment with learning rates to see if this gives a signficantly better reults
mod_20_50, res_20_50, y_test_20_50, dtest_20_50, X_test_20_50 = mod_shotid(df3[df3.ModGroup == "Mod"], 0.3, 34, 64, 95, 125, 156, 186, 217, 247, 278, 308, 339, 369 )

[0]	Test-merror:0.27027
[1]	Test-merror:0.22544
[2]	Test-merror:0.21490
[3]	Test-merror:0.21622
[4]	Test-merror:0.21490
[5]	Test-merror:0.21951
[6]	Test-merror:0.21490
[7]	Test-merror:0.21819
[8]	Test-merror:0.21226
[9]	Test-merror:0.20897
[10]	Test-merror:0.20435
[11]	Test-merror:0.20040
[12]	Test-merror:0.20369
[13]	Test-merror:0.20435
[14]	Test-merror:0.20171
[15]	Test-merror:0.20171
[16]	Test-merror:0.19974
[17]	Test-merror:0.20303
[18]	Test-merror:0.20369
[19]	Test-merror:0.20435
[20]	Test-merror:0.20105
[21]	Test-merror:0.20303
[22]	Test-merror:0.20105
[23]	Test-merror:0.20040
[24]	Test-merror:0.19974
[25]	Test-merror:0.20105
[26]	Test-merror:0.19842
[27]	Test-merror:0.19776
[28]	Test-merror:0.19710
[29]	Test-merror:0.19974
[30]	Test-merror:0.19842
[31]	Test-merror:0.19842
[32]	Test-merror:0.19842
[33]	Test-merror:0.20040
[34]	Test-merror:0.20105
[35]	Test-merror:0.20105
[36]	Test-merror:0.20040
[37]	Test-merror:0.20105
[38]	Test-merror:0.20237
[39]	Test-merror:0.20237
[40]	Test-

In [137]:
#experiment with learning rates to see if this gives a signficantly better reults
mod_20_50, res_20_50, y_test_20_50, dtest_20_50, X_test_20_50 = mod_shotid(df3[df3.ModGroup == "Mod"], 0.5, 34, 64, 95, 125, 156, 186, 217, 247, 278, 308, 339, 369 )

[0]	Test-merror:0.27027
[1]	Test-merror:0.24654
[2]	Test-merror:0.22544
[3]	Test-merror:0.21688
[4]	Test-merror:0.21688
[5]	Test-merror:0.21358
[6]	Test-merror:0.20962
[7]	Test-merror:0.20831
[8]	Test-merror:0.20765
[9]	Test-merror:0.20765
[10]	Test-merror:0.20897
[11]	Test-merror:0.20633
[12]	Test-merror:0.21094
[13]	Test-merror:0.20435
[14]	Test-merror:0.20303
[15]	Test-merror:0.20237
[16]	Test-merror:0.20105
[17]	Test-merror:0.20040
[18]	Test-merror:0.20171
[19]	Test-merror:0.20369
[20]	Test-merror:0.19908
[21]	Test-merror:0.20171
[22]	Test-merror:0.20435
[23]	Test-merror:0.20171
[24]	Test-merror:0.20171
[25]	Test-merror:0.19908
[26]	Test-merror:0.20040
[27]	Test-merror:0.19974
[28]	Test-merror:0.19974
[29]	Test-merror:0.19908
[30]	Test-merror:0.19974
[31]	Test-merror:0.20171
[32]	Test-merror:0.20303
[33]	Test-merror:0.20369
[34]	Test-merror:0.20303
[35]	Test-merror:0.20171
[36]	Test-merror:0.20171
[37]	Test-merror:0.20105
[38]	Test-merror:0.19974
[39]	Test-merror:0.20040
[40]	Test-

take an old model - which?
and build rules around it
 * first test on test, then validation, then decide on eval (model stacking)
Check that they roughly work out - use 20_50 + load an old model - don't need to build in the FH and Slice stuff... 
First check works generically and then can flex to make it make sense - tell story

Bringing in other models to see if the combination can enhance the prediction

In [36]:

#change df here to be shots_wide2 - also add in timetruestrike or how its defined
eval_fin = df3[["Who", "TimeTrueStrike", "origindex"]]

def gen_results_old(results, Name, dic):
        # results["Correct"]= np.where( results.Label3 == results.preds, 1, 0)
    eval_fin[f"{Name}_prob"] = results.max(axis=1)
    results['Max_Col'] = results.idxmax(axis=1)
    
    eval_fin[f"{Name}_pred"] = results.Max_Col.apply(lambda x: dic[x])
    return eval_fin

dic2 = {}
dic2["A"] = "BH"
dic2["B"] = "Slice"
dic2["C"] = "FH"
dic2["D"] = "Volley"
dic2["E"] = "OH"

# create the data so can predict on it
    #data is fine as is - just drop the extra cols at start
BHfocus_mod_cols = ['Acc_X_00', 'Acc_X_21', 'Acc_X_22', 'Acc_X_23', 'Acc_X_24', 'Acc_X_25', 'Acc_X_26', 'Acc_X_27',
 'Acc_X_28', 'Acc_X_29', 'Acc_X_30', 'Acc_X_31', 'Acc_X_32', 'Acc_X_33', 'Acc_X_34', 'Acc_X_35',
 'Acc_X_36', 'Acc_X_37', 'Acc_X_38', 'Acc_X_39', 'Acc_X_40', 'Acc_X_41', 'Acc_Y_21', 'Acc_Y_22',
 'Acc_Y_23', 'Acc_Y_24', 'Acc_Y_25', 'Acc_Y_26', 'Acc_Y_27', 'Acc_Y_28', 'Acc_Y_29', 'Acc_Y_30',
 'Acc_Y_31', 'Acc_Y_32', 'Acc_Y_33', 'Acc_Y_34', 'Acc_Y_35', 'Acc_Y_36', 'Acc_Y_37', 'Acc_Y_38',
 'Acc_Y_39', 'Acc_Y_40', 'Acc_Y_41', 'Acc_Z_21', 'Acc_Z_22', 'Acc_Z_23', 'Acc_Z_24', 'Acc_Z_25',
 'Acc_Z_26', 'Acc_Z_27', 'Acc_Z_28', 'Acc_Z_29', 'Acc_Z_30', 'Acc_Z_31', 'Acc_Z_32', 'Acc_Z_33',
 'Acc_Z_34', 'Acc_Z_35', 'Acc_Z_36', 'Acc_Z_37', 'Acc_Z_38', 'Acc_Z_39', 'Acc_Z_40', 'Acc_Z_41',
 'Gyr_X_21', 'Gyr_X_22', 'Gyr_X_23', 'Gyr_X_24', 'Gyr_X_25', 'Gyr_X_26', 'Gyr_X_27', 'Gyr_X_28',
 'Gyr_X_29', 'Gyr_X_30', 'Gyr_X_31', 'Gyr_X_32', 'Gyr_X_33', 'Gyr_X_34', 'Gyr_X_35', 'Gyr_X_36',
 'Gyr_X_37', 'Gyr_X_38', 'Gyr_X_39', 'Gyr_X_40', 'Gyr_X_41', 'Gyr_Y_21', 'Gyr_Y_22', 'Gyr_Y_23',
 'Gyr_Y_24', 'Gyr_Y_25', 'Gyr_Y_26', 'Gyr_Y_27', 'Gyr_Y_28', 'Gyr_Y_29', 'Gyr_Y_30', 'Gyr_Y_31',
 'Gyr_Y_32', 'Gyr_Y_33', 'Gyr_Y_34', 'Gyr_Y_35', 'Gyr_Y_36', 'Gyr_Y_37', 'Gyr_Y_38', 'Gyr_Y_39',
 'Gyr_Y_40', 'Gyr_Y_41', 'Gyr_Z_21', 'Gyr_Z_22', 'Gyr_Z_23', 'Gyr_Z_24', 'Gyr_Z_25', 'Gyr_Z_26',
 'Gyr_Z_27', 'Gyr_Z_28', 'Gyr_Z_29', 'Gyr_Z_30', 'Gyr_Z_31', 'Gyr_Z_32', 'Gyr_Z_33', 'Gyr_Z_34',
 'Gyr_Z_35', 'Gyr_Z_36', 'Gyr_Z_37', 'Gyr_Z_38', 'Gyr_Z_39', 'Gyr_Z_40', 'Gyr_Z_41']

mod_BH2040Focus = pickle.load(open("D:/OneDrive/DataSci/Tennis/03_Modelling/Models/ShotId_BH2040Focus_220126.pkl",'rb'))

probs = mod_BH2040Focus.predict(xgb.DMatrix(df3[BHfocus_mod_cols]))
eval_fin["BH_2040Focus_probs"] = probs
eval_fin["BH_2040Focus_preds"] = np.where(eval_fin.BH_2040Focus_probs>=0.2, "BH", "Other")

So it appears something went messy with my merges, and having fixed this now, I need to bring together the combo pred and add this new FH to it

So I need to look at where those old ComboPreds came from - they are not complete here...  as originally run on xsens only it appears  

Simple rule could be if new says FH, then FH, else other...   
Could be more I can do here... BH is very good. Find ways to make it work.

In [37]:
#combine the data so have all model predictions in one place
# df4 = pd.concat([df3, eval_fin.iloc[:,3:]], axis = 1)
df4 = pd.merge(df3, eval_fin[["BH_2040Focus_probs","BH_2040Focus_preds","origindex"]], 
                             how = "left", on = "origindex")

#show the quality of the prediction via confusion matrix
pd.crosstab(df4.CleanShot, df4.BH_2040Focus_preds)

BH_2040Focus_preds,BH,Other
CleanShot,,
BH,1734,99
FH,108,3390
OH,0,108
Slice,174,1066
Volley,56,292


In [43]:
# on basis of this, build a prediction of if BH from focus, else the rest
res_20_40_2 = pd.merge(res_20_40, df4[["BH_2040Focus_probs","origindex"]], how = "left", on = "origindex")
res_20_40_2["BH_FHPred"] = np.where(res_20_40_2.BH_2040Focus_probs > 0.2, "BH", res_20_40_2.Preds)
res_20_40_2["NewCombo_Correct"] = np.where(res_20_40_2.BH_FHPred == res_20_40_2.CleanShot, 1,0)
pd.crosstab(res_20_40_2.CleanShot, res_20_40_2.BH_FHPred)

BH_FHPred,BH,FH,OH,Slice,Volley
CleanShot,,,,,
BH,379,8,0,4,1
FH,38,725,0,68,8
OH,0,9,0,11,0
Slice,34,26,1,96,8
Volley,16,9,2,37,37


In [44]:
res_20_40_2.groupby("CleanShot")["NewCombo_Correct"].agg({"count", "sum", "mean"}).reset_index()[["CleanShot", "count", "sum", "mean"]]

,CleanShot,count,sum,mean
0,BH,392,379,0.966837
1,FH,839,725,0.864124
2,OH,20,0,0.000000
3,Slice,165,96,0.581818
4,Volley,101,37,0.366337


In [45]:
res_20_40_2.groupby("BH_FHPred")["NewCombo_Correct"].agg({"count", "sum", "mean"}).reset_index()[["BH_FHPred", "count", "sum", "mean"]]

,BH_FHPred,count,sum,mean
0,BH,467,379,0.811563
1,FH,777,725,0.933076
2,OH,3,0,0.000000
3,Slice,216,96,0.444444
4,Volley,54,37,0.685185


In [46]:
#To Do's then create validation data set in the mod line (then don't need to create them later - a more efficient way would be to use the list)
# I just need to make the predictions - cam create a function that gens the output I want so I can compare - 
#post predict - is this value right, cross tab and eval


valid_y_20_40, evalu_y_20_40 = gen_results(mod_20_40, X_test_20_40, df4[df4.ModGroup == "Validation"], df4[df4.ModGroup == "Eval"])

KeyError: "['BH_2040Focus_prob', 'BH_2040Focus_pred', 'Combined_OldPred', 'ServeProbs'] not in index"

In [38]:
#create the accuracy of the prediction - how many of the shots are correctly identified per shot
df4["FocusCorrect"] = np.where(df4.CleanShot == df4.BH_2040Focus_preds, 1,0)
df4.groupby("CleanShot")["FocusCorrect"].agg({"count", "sum", "mean"}).reset_index()[["CleanShot", "count", "sum", "mean"]]

,CleanShot,count,sum,mean
0,BH,1833,1734,0.94599
1,FH,3498,0,0.00000
2,OH,108,0,0.00000
3,Slice,1240,0,0.00000
4,Volley,348,0,0.00000


In [41]:
#understand the quality of the predictions - ie how trustworthy is a prediction
df4.groupby("BH_2040Focus_preds")["FocusCorrect"].agg({"count", "sum", "mean"}).reset_index()[["count", "sum", "mean"]]

,count,sum,mean
0,2072,1734,0.836873
1,4955,0,0.000000


In [17]:
# I'm searching for a rule where can downplay slice using older model: take favourite model from above and compare modTest

#add in the prior model predctions to this test data set
res_20_40_2 = pd.merge(res_20_40, pd.concat([df4[["origindex"]], df4.iloc[:,-6:]], axis = 1), how = "left", on = "origindex")
res_20_40_2["BH_FHPred"] = np.where(res_20_40_2.BH_2040Focus_prob > 0.2, "BH", res_20_40_2.Preds)
res_20_40_2["NewCombo_Correct"] = np.where(res_20_40_2.BH_FHPred == res_20_40_2.CleanShot, 1,0)
pd.crosstab(res_20_40_2.CleanShot, res_20_40_2.BH_FHPred)

Combined_OldPred,BH,FH,OH,Slice,Volley
CleanShot,,,,,
BH,379,3,0,10,0
FH,187,275,9,326,19
OH,1,5,0,12,2
Slice,57,54,0,44,3
Volley,24,20,0,50,3


In [19]:
pd.crosstab(res_20_40_2.CleanShot, res_20_40_2.Preds)

Preds,BH,FH,OH,Slice,Volley
CleanShot,,,,,
BH,337,30,1,21,3
FH,29,729,0,73,8
OH,0,9,0,11,0
Slice,20,26,1,110,8
Volley,9,9,2,43,38


In [22]:
#take BH where focus says it is
pd.crosstab(res_20_40_2.CleanShot, res_20_40_2.BH_2040Focus_pred)

BH_2040Focus_pred,BH,Other
CleanShot,,
BH,379,13
FH,38,801
OH,0,20
Slice,31,134
Volley,15,86


In [24]:
rtest = res_20_40_2[res_20_40_2.CleanShot != "BH"]
pd.crosstab(rtest.CleanShot, rtest.Preds)

Preds,BH,FH,OH,Slice,Volley
CleanShot,,,,,
FH,29,729,0,73,8
OH,0,9,0,11,0
Slice,20,26,1,110,8
Volley,9,9,2,43,38


In [27]:
pd.crosstab(rtest.CleanShot, rtest.Combined_OldPred)

Combined_OldPred,BH,FH,OH,Slice,Volley
CleanShot,,,,,
FH,187,275,9,326,19
OH,1,5,0,12,2
Slice,57,54,0,44,3
Volley,24,20,0,50,3


In [31]:
rtest[rtest.CleanShot.isin(["FH", "Slice"])]

,index,CleanShot,Label_Num_x,Who,origindex,A,B,C,D,E,Max_Col,Max,Preds,Correct,ModTestSplit,Label_Num_y,BH_2040Focus_prob,BH_2040Focus_pred,Combined_OldPred,OldCorrect
1,3385,FH,2,Leo,3385,0.002105,0.001238,0.994766,0.000770,0.001120,C,0.994766,FH,1,modTest,2,0.022964,Other,FH,1
3,3387,FH,2,Leo,3387,0.001412,0.005239,0.990581,0.001298,0.001470,C,0.990581,FH,1,modTest,2,0.024195,Other,FH,1
5,3389,FH,2,Leo,3389,0.003726,0.013581,0.971540,0.004171,0.006982,C,0.971540,FH,1,modTest,2,0.046007,Other,Slice,0
6,3390,Slice,1,Leo,3390,0.152888,0.833044,0.005933,0.004372,0.003764,B,0.833044,Slice,1,modTest,1,0.148870,Other,FH,0
7,3391,FH,2,Leo,3391,0.001759,0.007595,0.985233,0.002136,0.003277,C,0.985233,FH,1,modTest,2,0.048139,Other,BH,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,8411,FH,2,TL,8411,0.002022,0.004240,0.991701,0.000966,0.001071,C,0.991701,FH,1,modTest,2,0.033595,Other,NaN,0
1512,8416,FH,2,TL,8416,0.001756,0.002167,0.993980,0.000927,0.001170,C,0.993980,FH,1,modTest,2,0.019367,Other,NaN,0
1513,8417,Slice,1,TL,8417,0.120910,0.796850,0.038259,0.032468,0.011513,B,0.796850,Slice,1,modTest,1,0.041488,Other,NaN,0
1514,8418,FH,2,TL,8418,0.002031,0.003126,0.991289,0.001299,0.002254,C,0.991289,FH,1,modTest,2,0.033683,Other,NaN,0


OK, so we take BH first and the newest one here... can clean up prior bring in

Then look at where it struggled - on this data.

In [29]:
res_20_40_2["BH_FHPred"] = np.where(res_20_40_2.BH_2040Focus_prob > 0.2, "BH", res_20_40_2.Preds)
res_20_40_2["NewCombo_Correct"] = np.where(res_20_40_2.BH_FHPred == res_20_40_2.CleanShot, 1,0)
pd.crosstab(res_20_40_2.CleanShot, res_20_40_2.BH_FHPred)

BH_FHPred,BH,FH,OH,Slice,Volley
CleanShot,,,,,
BH,379,8,0,4,1
FH,38,725,0,68,8
OH,0,9,0,11,0
Slice,34,26,1,96,8
Volley,16,9,2,37,37


In [30]:
#understand the class accuracy
res_20_40_2.groupby("CleanShot")["NewCombo_Correct"].agg({"count", "sum", "mean"})

,mean,sum,count
CleanShot,,,
BH,0.966837,379,392
FH,0.864124,725,839
OH,0.000000,0,20
Slice,0.581818,96,165
Volley,0.366337,37,101
